In [89]:
import base64
import datetime
from urllib.parse import urlencode

import requests

client_id = '24b72d4503684819a52f85b3e67a935e'
client_secret = '2eb51fdc122d41659b640e1bd879e119'

In [91]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = 'https://accounts.spotify.com/api/token'

    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        self.client_id = client_id
        self.client_secret = client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must enter client_id and secret")

        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()

    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200,299):
            raise Exception('Could not authenticate client')
            return False
        data = r.json()
        now = datetime.datetime.now()
        access_token =  data['access_token']
        expires_in = data['expires_in']
        expires = now + datetime.timedelta(seconds = expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        access_token_did_expire = expires < now
        return True
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
    'Authorization': f'Basic {client_creds_b64}'
}

    def get_token_data(self):
        return {
    'grant_type':'client_credentials'

}

    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token

    def base_search(self, query_params):
        headers = self.get_resource_headers()

        endpoint = 'https://api.spotify.com/v1/search'
        
        lookup_url = f'{endpoint}?{query_params}'

        r = requests.get(lookup_url, headers=headers)
        
        if r.status_code not in range(200,299):
            return {}
        return r.json()

    def search(self, query=None, operator=None, operator_query=None, search_type='artist'):
        if query == None:
            raise Exception('Query required')
        if isinstance(query, dict):
            query = " ".join([f'{k}:{v}' for k,v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == 'or' or operator.lower()=='not':
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f"{query} {operator} {operator}"

        query_params = urlencode({'q': query, 'type': search_type.lower()})

        return self.base_search(query_params)

    
    def get_resource_headers(self):
        access_token = self.get_access_token()
        headers = {
            'Authorization':f'Bearer {access_token}'
        }
        return headers


    
    def get_resource(self, lookup_id, resource_type='albums', version = 'v1'):
        endpoint = f'https://api.spotify.com/{version}/{resource_type}/{lookup_id}'
        headers = self.get_resource_headers()
        r = requests.get(endpoint, headers = headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()

    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')


    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')

In [94]:
type(spotify.search('Time', search_type = 'track'))

dict

In [104]:
spotify.search(query='Sam Wise', operator='NOT', operator_query='Zone', search_type='artist')

{'artists': {'href': 'https://api.spotify.com/v1/search?query=Sam+Wise+NOT+Zone&type=artist&offset=0&limit=20',
  'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4CCjWYtowoZyWAl03Iih5P'},
    'followers': {'href': None, 'total': 15772},
    'genres': ['uk alternative hip hop'],
    'href': 'https://api.spotify.com/v1/artists/4CCjWYtowoZyWAl03Iih5P',
    'id': '4CCjWYtowoZyWAl03Iih5P',
    'images': [{'height': 640,
      'url': 'https://i.scdn.co/image/f3e33c752f9b074bdf3878f08dbca140c26bcfc4',
      'width': 640},
     {'height': 320,
      'url': 'https://i.scdn.co/image/7b46ef8c21063bea19a3ecea9e6b6adfdaf3d00b',
      'width': 320},
     {'height': 160,
      'url': 'https://i.scdn.co/image/f08513905d322493ac048cc6ac6f09e9b5526d52',
      'width': 160}],
    'name': 'Sam Wise',
    'popularity': 52,
    'type': 'artist',
    'uri': 'spotify:artist:4CCjWYtowoZyWAl03Iih5P'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/6KQyr4Kv5Gu2Qj69uFHu